In [1]:
import os
# Display current working directory
print(os.getcwd())
# To make sure opencv imports .exr files
os.environ["OPENCV_IO_ENABLE_OPENEXR"] = "1"
# If the current directory is not WDSS, then set it to one level up
if os.getcwd()[-4:] != 'WDSS':
    os.chdir('..')
print(os.getcwd())

d:\College\III_year_II_part\WDSS\jupyter_notebooks
d:\College\III_year_II_part\WDSS


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import numpy as np
import cv2
import matplotlib.pyplot as plt
import time

from typing import List, Tuple, Dict

from config import device, Settings
from commons import initialize

In [3]:
settings = Settings("config/config.json", "WDSSV5")
initialize(settings=settings)

Job: prashant_pt_nonormal, Model: WDSSV5, Device: cuda
Model path: out\prashant_pt_nonormal-WDSSV5\model
Log path: out\prashant_pt_nonormal-WDSSV5\logs


In [4]:
from network.dataset import *

train_dataset, val_dataset, test_dataset = WDSSDatasetCompressed.get_datasets(settings)

In [5]:
from network.models.WDSS import get_wdss_model
    
# Model
model = get_wdss_model(settings.model_config).to(device)

In [ ]:
from network.losses import Criterion_Combined

criterion = Criterion_Combined({
                'l1': 0.5,
                'ssim': 0.05, 
                'lpips': 0.05,
                'l1_wave': 0.5,
                'ssim_wave': 0.05
            })
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = StepLR(optimizer, step_size=20, gamma=0.5)

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


d:\College\III_year_II_part\WDSS\.conda\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\College\III_year_II_part\WDSS\.conda\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: d:\College\III_year_II_part\WDSS\.conda\Lib\site-packages\lpips\weights\v0.1\vgg.pth
Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


d:\College\III_year_II_part\WDSS\.conda\Lib\site-packages\lpips\lpips.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(model_path, map_l

Loading model from: d:\College\III_year_II_part\WDSS\.conda\Lib\site-packages\lpips\weights\v0.1\vgg.pth


In [7]:
from torch.utils.data import DataLoader

In [8]:
data_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=0)

for batch in data_loader:
    hr = batch["HR"].to(device)
    wavelet = WaveletProcessor.batch_wt(hr)
    hr_2 = hr.clone()
    wavelet_2 = WaveletProcessor.batch_wt(hr_2)
    loss, _ = criterion.forward(wavelet, wavelet_2, hr, hr_2)

    print(loss)
    break

tensor(0., device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
